# 01. BERT Overview & Transformer 맛보기
### 과목: NLP Transfer Learning
---
**학습 목표**
1. **BERT**가 무엇인지, 기존 모델과 무엇이 다른지 이해한다.
2. **Tokenizer**의 역할과 Special Token(`[CLS], [SEP], [MASK]`)을 이해한다.
3. **Masked Language Model(MLM)**과 **Next Sentence Prediction(NSP)**가 어떻게 학습되는지 코드로 실습한다.

# 1. BERT란 무엇인가?

**BERT**(Bidirectional Encoder Representations from Transformers)는 구글이 개발한 자연어 처리 모델입니다.

### 🍎 쉬운 비유: "언어 천재 김AI"
- 옛날 모델(GPT-1 등): 책을 왼쪽에서 오른쪽으로만 읽을 수 있었어요. (단방향)
- **BERT**: 책을 한 번에 펼쳐서 **문장의 앞뒤 문맥을 동시에** 봅니다. (양방향, Bidirectional)
- 그래서 빈칸 채우기(`[MASK]`)나 문장의 의미 파악을 엄청나게 잘합니다.

### BERT의 3가지 핵심 특징
1. **양방향성 (Bidirectionality):** 문맥을 양쪽에서 파악하므로 "배"가 먹는 배인지, 타는 배인지 정확히 알 수 있음.
2. **Transformer Encoder:** 문장을 이해하는(Encoding) 능력이 탁월한 Transformer의 인코더 부분만 사용.
3. **전이 학습 (Transfer Learning):** 대량의 데이터로 사전 학습(Pre-training)된 모델을 가져와서, 내 목적에 맞게 미세 조정(Fine-tuning)하여 사용.

### 2. 준비물 챙기기 (라이브러리 로드)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Hugging Face Transformers 라이브러리 설치 (없다면 실행)
# !pip install transformers -q

# 3. Tokenizer (토크나이저) 이해하기

컴퓨터는 '사과', '학교' 같은 글자를 이해하지 못합니다. 숫자로 바꿔줘야 하죠.
이 역할을 하는 것이 **Tokenizer**입니다.

- **역활:** 문장 -> 조각(Token) -> 숫자(ID)
- **WordPiece:** BERT는 단어를 더 작은 단위(Sub-word)로 쪼개서, 모르는 단어(OOV)가 나와도 대처할 수 있게 합니다.

In [ ]:
from transformers import BertTokenizer

model_name = 'bert-base-uncased' # 'uncased'는 소문자로만 이루어진 모델이라는 뜻입니다.

# 1. 사전 학습된(Pre-trained) BERT의 토크나이저(단어장+규칙)를 불러옵니다.
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer

In [ ]:
# 2. 문장을 숫자로 바꿔봅시다. (Encoding)
text = 'Here is the sentence I want embedding for.'

# tokenizer.encode() -> 자동으로 [CLS], [SEP] 같은 특수 토큰을 붙여서 정수 ID로 변환해줍니다.
encoded_ids = tokenizer.encode(text)
print("Encoded IDs:", encoded_ids)

In [ ]:
# 3. 실제로 어떻게 쪼개졌는지 눈으로 확인해볼까요? (Tokenize)
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# 'embedding'이라는 단어가 'em', '##bed', '##ding'으로 쪼개진 것을 볼 수 있습니다.
# '##'은 앞 단어에 붙는다는 뜻입니다. (Sub-word Tokenization)

In [ ]:
# 4. 모델에 넣기 좋은 종합 선물 세트 (tokenizer())
# input_ids, token_type_ids, attention_mask를 한 번에 만들어줍니다.
inputs = tokenizer(text)
inputs

# 4. Masked Language Model (MLM): 빈칸 채우기 놀이

BERT가 똑똑해진 비결입니다.
입력 문장의 일부를 `[MASK]`로 가리고, 그 자리에 원래 어떤 단어가 있었는지 맞히게 합니다.

> 예시: "축구는 정말 재밌는 **[MASK]**다." -> 정답: 스포츠

In [ ]:
from transformers import BertForMaskedLM

# Masked LM 전용 헤드(Head)가 달린 BERT 모델을 불러옵니다.
model = BertForMaskedLM.from_pretrained(model_name)
model

In [ ]:
# 1. 실습할 문장 준비 ([MASK] 포함)
text = 'Soccer is a really fun [MASK].'

# 2. 모델에 입력할 수 있도록 텐서(Tensor) 형태로 변환 (return_tensors='pt')
inputs = tokenizer(text, return_tensors='pt')

print("입력 토큰:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
inputs

In [ ]:
# 참고: BERT가 사용하는 특수 토큰의 ID 확인
print('CLS (문장 시작): ', tokenizer.cls_token_id)
print('SEP (문장 끝): ', tokenizer.sep_token_id)
print('MASK (빈칸): ', tokenizer.mask_token_id)

In [ ]:
# 3. 모델에 넣고 예측하기 (Forward)
output = model(**inputs)

# logits: 모델이 3만 개의 단어장 중에서 각 단어가 정답일 점수를 계산한 값
print("출력 크기 (Batch, Seq_len, Vocab_size):", output.logits.shape)

### 🤖 더 쉽게 예측하기: Pipeline 사용
코드로 일일이 logits에서 argmax를 뽑는 건 귀찮죠? Hugging Face의 **Pipeline**을 쓰면 한 방에 해결됩니다.

In [ ]:
from transformers import FillMaskPipeline

# 빈칸 채우기(Fill-Mask) 전용 파이프라인 생성
pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [ ]:
# 결과 확인: "Soccer is a really fun [MASK]."
# score가 높은 순서대로 Top 5 후보를 보여줍니다.
pipe(text)

In [ ]:
# 다른 예시도 해볼까요?
pipe("I went to [MASK] this morning.")

# 5. AutoClass (AutoTokenizer, AutoModel)

"매번 `BertTokenizer`, `RobertaTokenizer` 이름을 다 외워야 하나요?"
아닙니다! `AutoTokenizer`를 쓰면 모델 이름만 보고 알아서 딱 맞는 클래스를 가져옵니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name = 'bert-base-uncased'

# Auto 클래스 사용 (추천하는 방법!)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

print("실제 로드된 토크나이저:", tokenizer.__class__.__name__)
print("실제 로드된 모델:", model.__class__.__name__)

# 6. Next Sentence Prediction (NSP): 문장 이어짓기 판별

BERT는 **두 문장** 사이의 관계도 이해합니다.
문장 A 다음에 문장 B가 오는 게 자연스러운지 아닌지(True/False)를 맞히는 문제입니다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForNextSentencePrediction

# 이번엔 NSP 헤드가 달린 모델을 가져옵니다.
model = AutoModelForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# 두 문장을 준비합니다.
sentence1 = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
sentence2 = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

# 두 문장을 같이 넣어주면, 자동으로 [SEP] 토큰으로 구분해줍니다.
inputs = tokenizer(sentence1, sentence2, return_tensors='pt')
inputs

### 👀 입력이 어떻게 들어가는지 시각화

In [ ]:
# [CLS] 문장1 [SEP] 문장2 [SEP] 구조인지 확인해봅시다.
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print(tokens[:20]) # 앞부분만 출력

In [ ]:
import torch.nn.functional as F
import torch

# 모델 예측
output = model(**inputs)

# 결과는 0(이어짐) 또는 1(안 이어짐)로 나옵니다. (모델마다 인덱스 의미가 다를 수 있음)
# bert-base-uncased: 0 = IsNext (이어짐), 1 = NotNext (안 이어짐)
probs = F.softmax(output.logits, dim=-1)
print("확률:", probs)

pred = torch.argmax(probs, dim=-1)
print("예측값 (0이면 이어짐):", pred.item())

In [ ]:
# 엉뚱한 문장을 넣어볼까요?
sentence3 = "The sky is blue due to the shorter wavelength of blue light."

inputs = tokenizer(sentence1, sentence3, return_tensors='pt')
output = model(**inputs)

probs = F.softmax(output.logits, dim=-1)
pred = torch.argmax(probs, dim=-1)

print("예측값 (1이면 안 이어짐):", pred.item())

### 🎉 축하합니다!
이제 여러분은 BERT가 '단어'를 어떻게 이해하고(Masked LM), '문장'을 어떻게 연결하는지(NSP) 완벽하게 이해했습니다.